<a href="https://colab.research.google.com/github/carlibeisel/pod_pou_lulcc/blob/main/BPBC_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Climate + LULCC for Boise Project

##LULCC##

In [1]:
## IMPORT PACKAGES ##
import numpy as np #basic computation
!pip install geopandas
import geopandas as gpd #geopandas for .shp
import matplotlib.pyplot as plt #to create plots
import pandas as pd #to create dataframes and export .csv
!pip install rasterio
import rasterio as rso #import GeoTiff files
from rasterio.mask import mask #to crop data to a boundary
from rasterio.plot import show #to plot the image
from rasterio.crs import CRS
from shapely.ops import unary_union #creates boundary of shapefile
import json #imports metadata
!pip install rioxarray #to clip rasters to a .shp file
import rioxarray as rxr
from rasterio.warp import calculate_default_transform, reproject, Resampling
!pip install pylandstats==2.1.3 #very important
import pylandstats #to perform landscape metrics
from pylandstats import landscape
from pylandstats import SpatioTemporalAnalysis #to calculate landscape metrics through time
import glob
import os
import matplotlib.lines as lines
import matplotlib.patches as patch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 970.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 896.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.2 MB/s eta 0:00:00


WARNING  Module /usr/local/lib/python3.10/dist-packages/pylandstats/landscape.py has not been compiled for         
         Transonic-Pythran

In [2]:
#Navigate to your directory

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
## ------------------------------ ##
## SUBSET GEOSPATIAL DATA TO BPBC ##
## ------------------------------ ##

shp_file = gpd.read_file('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/BPBC_Divisions/BPBC_Divisions-IDTM.shp') #open shapefile
names = shp_file['DivisionNa']
files = glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/*.tiff') #get all the years of cdl imagery
data =[]
for i in range(len(files)):
  data.append(rso.open(files[i])) #open cdl image and append to a list
shp = shp_file.to_crs(data[1].crs) #reproject the shp file to same projection
years = np.arange(1987, 2021) #years of LCMAP data
collection = []
for i in range(len(shp)):
  for n in range(len(years)):
    dataset = data[n]
    year_out = dataset.name[21:-15]
    extent = gpd.GeoSeries(shp['geometry'][i]) #get the geometry from shapefile
    coords = [json.loads(extent.to_json())['features'][0]['geometry']] #gets coordinates for rasterio input
    out_img, out_transform = mask(dataset=data[n], shapes=coords, crop=True, nodata=0) #crop the data to the shapefile
    out_meta = data[n].meta.copy()
    out_meta.update({"driver": "GTiff",
                     "height": out_img.shape[1],
                     "width": out_img.shape[2],
                     "transform": out_transform})
    # Merge original file name with init_landcover to denote that it is the initial land cover data for Janus
    in_file = files[n]
    out_filename = os.path.join('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmap_masked_bpbc/'+names[i]+'_'+year_out+'.tif') #create a file name to export to
    # Save clipped land cover coverage THIS WILL OVERWRITE FILES
    out_tiff = rasterio.open(out_filename, 'w', **out_meta)
    out_tiff.write(np.squeeze(out_img, 0), 1)
    out_tiff.close()
    collection.append(out_img)

RasterioIOError: Attempt to create new tiff file '/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmap_masked_bpbc/NAMPA & MERIDIAN_e/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmaps/LCMAP_CU_1987.tif' failed: No such file or directory

In [ ]:
## ---------------------------------------- ##
## Import multiple rasters into PyLandStats ##
## ---------------------------------------- ##
years = np.arange(1987,2021)
temporal_group = []
for i in range(len(names)):
  files= sorted(glob.glob('lcmap_files/lcmap_masked_bpbc/'+names[i]+'_*.tif')) #name for all the csv files
  sta = SpatioTemporalAnalysis(files, dates=years, nodata=0) #import all CDL rasters and mask
  temporal_group.append(sta)

In [ ]:
# ------------------------------------ #
# CALCULATE THE CLASS PROPORTIONS BPBC #
# ------------------------------------ #

proportions = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_class_metrics_df(temporal_group[i], metrics=['proportion_of_landscape'])
  df.to_csv('lcmap_files/proportions/bpbc_metrics/long/'+names[i]+'_prop.csv')
  proportions.append(df)

In [ ]:
# ------------------------------- #
#  CALCULATE LARGEST PATCH METRICS
# ------------------------------- #
patch = []

for i in range(len(names)):
  df = SpatioTemporalAnalysis.compute_landscape_metrics_df(temporal_group[i], metrics = ['largest_patch_index'])
  df.to_csv('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/largest_patch_index/'+names[i]+'_patch.csv')
  patch.append(df)

In [ ]:
# ------------------------------- #
#     CALCULATE CONTAGION
# ------------------------------- #
# Original code computed contagion + largest_patch_index in the same step. BUt you
# cannot compute contagion with landscape_metrics.
# Make sure you have the correct version of PyLandStats (2.1.3) or it won't run,
# the error code won't look like its a package version issue.
for i in range(len(names)):
    contag = []
    files = sorted(glob.glob('/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/lcmap_masked/'+names[i]+'_*.tif'))
    k = 0
    for file in files:
        sta = pylandstats.Landscape(file, nodata=0)
        DD = sta.contagion()
        output_file = '/content/gdrive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/contagion/' + names[i] + '_contagion.csv'
        contag.append({'dates': years[k], 'contagion': DD})
        k = k+1
        contag_df = pd.DataFrame(contag)
        contag_df.to_csv(output_file, index=False)

In [ ]:
# ----------------------------------------------------------------- #
# Put class proportions in the same format as configuration metrics #
# ----------------------------------------------------------------- #

# Import csv files into a list of dataframes
files_proportions = sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/proportions/*_prop.csv'))
files_contagion = sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/contagion/*_contagion.csv'))
files_patch = sorted(glob.glob('/content/drive/MyDrive/Data/pod_pou_lulcc/data_input/subset_LULCC/largest_patch_index/*_patch.csv'))
names = list(sorted(shp_file['WaterRight']))

proportions = []
for i in files_proportions:
  data = pd.read_csv(i)
  proportions.append(data)
contagion = []
for i in files_contagion:
  data = pd.read_csv(i)
  contagion.append(data)
patch = []
for i in files_patch:
  data = pd.read_csv(i)
  patch.append(data)

#Create new dataframes in same format as configuration metrics

new_df = []
for i in range(len(proportions)):
  df = pd.DataFrame(years, columns=['dates'])
  prop = proportions[i]
  df['DivName'] = names[i]
  df['class1_urban'] = prop['proportion_of_landscape'][prop['class_val'] == 1]
  df['class2_crops'] = prop['proportion_of_landscape'][prop['class_val'] == 2].values
  df = df.fillna(0)
  new_df.append(df)

In [ ]:
## ----------------------------------------------- ##
## CALCULATE CHANGE IN URBAN AREA FOR MAPPING BPBC ##
## ----------------------------------------------- ##

prop = pd.concat(new_df)

change = prop.groupby('DivName', as_index=False).class1_urban.agg(['min','max']).reset_index().fillna(0)
change['urb_change'] = change['max']-change['min']
change.to_csv('lcmap_files/proportions/bpbc_change.csv')


In [ ]:
## -------------------- ##
## MERGE DATAFRAMES ##
## -------------------- ##

merged = []

for i in range(len(new_df)):
  df = new_df[i]
  contag = contagion[i]
  pat = patch[i]
  df_merge = df.merge(contag, on='dates', how='left').merge(pat, on='dates', how='left')
  display (df_merge)
  df_merge.to_csv('/content/drive/MyDrive/Data/pod_pou_lulcc/data_output/subset_LULCC_out/final_metrics/'+ names[i] +'.csv')
  merged.append(df_merge)

## CLIMATE ##

In [ ]:
# Installs geemap package
import subprocess
!pip install geemap #added

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet') #input and create project on google earth engiine

In [ ]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## ------------------------------------------------- ##
## IMPORT PRECIP, TEMP, and ET FOR IRRIGATION SEASON ##
## ------------------------------------------------- ##

## BPBC ##

years = np.arange(1987,2021)

# Empty lists to store images
ir_tmp = []
ir_pr = []
mean_max = []
et_irrig = []

# Import image collection, subset to shapefile, take a statistics for a period of time,
# and append the image to the designated list
for i in range(len(start_end)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_end['StartDate'][i], start_end['EndDate'][i]) #get image collection for irrigation season
  daymet_hot = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection for June-Aug
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-04-01'), str(years[i])+'-10-31')
  et = et_data.map(lambda image: image.clip(bpbc)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters
  mxtmp = daymet_hot.select('tmax').map(lambda image: image.clip(bpbc)).mean().set({'system:index':str(years[i])}) #select temp to analyze hot months and take mean
  tmp = daymet.select('tmax').map(lambda image: image.clip(bpbc)).mean().set({'system:index':str(start_end['StartDate'][i])}) #select max temp to analyze and take mean
  pr = daymet.select('prcp').map(lambda image: image.clip(bpbc)).sum().set({'system:index': str(start_end['StartDate'][i])}) #select precip to analyze and sum
  ir_tmp.append(tmp)
  ir_pr.append(pr)
  mean_max.append(mxtmp)
  et_irrig.append(et)

# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
ir_pr = ee.ImageCollection(ir_pr)
means_max_temp = ee.ImageCollection(mean_max)

In [ ]:
## ---------------------------------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON ##
## ---------------------------------------------------------------------- ##

## BPBC ##
# This will provide insight into antecedent moisture conditions for a POU.

years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-31'), start_end['StartDate'][i]) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(bpbc)).sum().set({'system:index':str(i+1)}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map


In [ ]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# BPBC #

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('climate_stats/JAtemp_stats_bpbc.csv')
emap.zonal_statistics(means_max_temp, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ir_tmp_stats_bpbc.csv')
emap.zonal_statistics(ir_tmp, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/irrig_precip_stats_bpbc.csv')
emap.zonal_statistics(ir_pr, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/ant_precip_stats_bpbc.csv')
emap.zonal_statistics(ant_precip, bpbc, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('climate_stats/et_bpbc.csv')
emap.zonal_statistics(et_irrig, bpbc, out_stats, statistics_type='MEAN', scale=30)

In [ ]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

## BPBC ##

years = np.arange(1987,2021)
ir_precip = pd.read_csv('climate_stats/irrig_precip_stats_bpbc.csv')
ant_precip = pd.read_csv('climate_stats/ant_precip_stats_bpbc.csv')
JA_temp = pd.read_csv('climate_stats/JAtemp_stats_bpbc.csv')
irrig_temp = pd.read_csv('climate_stats/ir_tmp_stats_bpbc.csv')
et_irrig = pd.read_csv('climate_stats/et_bpbc.csv')

names = et_irrig['DivisionNa']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  df['irrig_prcp'] = ir_precip.iloc[i,0:34].values
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  out_path = os.path.join('climate_stats/bpbc_final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


In [ ]:
## BPBC Merging ##

relates = pd.read_csv('diversion_timeseries/bpbc/bpbc_relate.csv')

# Dicharge data dict
key_list = list(bpbc['DiversionNa'])
dict_lookup = dict(zip(relates['Discharge'], relates['NewName']))
bpbc['Name'] = [dict_lookup[item] for item in key_list]

# Land use change dict
key_list2 = list(land_bpbc['DivName'])
dict_lookup2 = dict(zip(relates['Shape'], relates['NewName']))
land_bpbc['Name'] = [dict_lookup2[item] for item in key_list2]

key_list3 = list(climate_bpbc['DIV_NAME'])
dict_lookup3 = dict(zip(relates['Shape'], relates['NewName']))
climate_bpbc['Name'] = [dict_lookup2[item] for item in key_list2]

## Flow data
bpbc = bpbc.drop('DiversionNa', axis=1)
div_bpbc = pd.concat([div, bpbc])
all_div = pd.DataFrame(div_bpbc[['Year', 'Name', 'Acre_feet']])
all_div = all_div.sort_values(by=['Name', 'Year'])

## Land use data
land_bpbc = land_bpbc.drop(['Unnamed: 0', 'DivName'], axis=1)
all_land = pd.concat([land_bpbc,land])

## Climate data
climate_bpbc = climate_bpbc.drop(['Unnamed: 0','DIV_NAME'], axis=1)
all_clim = pd.concat([climate_bpbc, clim])

In [ ]:
## ------------------------------------ ##
## MERGE THREE FILES INTO ONE FILE BPBC ##
## ------------------------------------ ##

land_div = all_div.merge(all_land, left_on=['Year', 'Name'], right_on=['dates','Name'], how='left')
full_df = land_div.merge(all_clim, left_on=['Year','Name'], right_on=['Year', 'Name'], how='left').sort_values(by=['Name', 'Year'])
full_df = full_df.merge(hydromet, left_on='Year', right_on='Year', how='left').drop(['Unnamed: 0', 'dates'], axis=1)

# Get rid of New York data because using BPBC data
full_df = full_df[full_df['Name'] != 'New York Canal']
print(full_df['Name'].unique())
display(full_df)
## --------------------------------------- ##
## Export the full csv file for model in R ##
## --------------------------------------- ##

# Full dataframe export
out_path = 'output_files/merged/bpbc_model_input.csv'
full_df.to_csv(out_path)
